In [ ]:
import tensorflow as tf

# This will list all physical devices TensorFlow can see
devices = tf.config.list_physical_devices('GPU')

if len(devices) > 0:
    print(f"✅ GPU DETECTED: {len(devices)} device(s) found.")
    print(devices)
else:
    print("❌ GPU NOT DETECTED. TensorFlow will use the CPU.")

In [ ]:
import numpy, scipy, sklearn
print("NumPy:", numpy.__version__)
print("SciPy:", scipy.__version__)
print("Scikit-learn:", sklearn.__version__)


In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error
import math

# ---------------------------
# 1. Load Stock Data
# ---------------------------
stock_symbol = 'AAPL'  # You can change to TSLA, MSFT, etc.
data = yf.download(stock_symbol, start='2018-01-01', end='2025-01-01')

# Use only the 'Close' price for simplicity
close_prices = data['Close'].values.reshape(-1, 1)

# ---------------------------
# 2. Preprocess Data
# ---------------------------
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_prices)

# 80% training, 20% testing split
training_data_len = int(len(scaled_data) * 0.8)
train_data = scaled_data[:training_data_len]
test_data = scaled_data[training_data_len - 60:]  # 60 for lookback

# ---------------------------
# 3. Create Sequences for LSTM
# ---------------------------
def create_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(time_step, len(dataset)):
        X.append(dataset[i - time_step:i, 0])
        y.append(dataset[i, 0])
    return np.array(X), np.array(y)

X_train, y_train = create_dataset(train_data)
X_test, y_test = create_dataset(test_data)

# Reshape to [samples, time steps, features]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# ---------------------------
# 4. Build the LSTM Model
# ---------------------------
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(100, return_sequences=False),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# ---------------------------
# 5. Train the Model
# ---------------------------
history = model.fit(X_train, y_train, batch_size=64, epochs=25, validation_split=0.1, verbose=1)

# ---------------------------
# 6. Make Predictions
# ---------------------------
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# ---------------------------
# 7. Evaluate Model
# ---------------------------
rmse = math.sqrt(mean_squared_error(y_test_actual, predictions))
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')

# ---------------------------
# 8. Visualize Results
# ---------------------------
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(12, 6))
plt.title(f'{stock_symbol} Stock Price Prediction (AlphaWave LSTM)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.plot(train['Close'], label='Training Data')
plt.plot(valid['Close'], label='Actual Price', color='blue')
plt.plot(valid['Predictions'], label='Predicted Price', color='red')
plt.legend()
plt.show()

# ---------------------------
# 9. Predict Next 30 Days (Optional)
# ---------------------------
last_60_days = scaled_data[-60:]
next_input = np.array(last_60_days).reshape(1, 60, 1)
next_30_days = []

for _ in range(30):
    next_price = model.predict(next_input)[0, 0]
    next_30_days.append(next_price)
    next_input = np.append(next_input[:, 1:, :], [[[next_price]]], axis=1)

# Inverse scale predictions
next_30_days = scaler.inverse_transform(np.array(next_30_days).reshape(-1, 1))

# Plot future prediction
plt.figure(figsize=(10, 5))
plt.plot(range(1, 31), next_30_days, marker='o', color='green')
plt.title(f'{stock_symbol} - Predicted Prices for Next 30 Days (AlphaWave)')
plt.xlabel('Days Ahead')
plt.ylabel('Predicted Price (USD)')
plt.grid()
plt.show()


In [ ]:
# AlphaWave v2 – Universal Multi-Stock LSTM Predictor (with model saving)
# Author: AriseAK

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error
import joblib  # for saving scaler
import math
import os

# ---------------------------
# 1. Configuration
# ---------------------------
stock_symbols = [
    'AAPL', 'MSFT', 'TSLA', 'GOOG', 'AMZN', 'NVDA', 'META', 'NFLX', 'AMD', 'INTC',
    'ADBE', 'ORCL', 'IBM', 'PYPL', 'CRM', 'CSCO', 'PEP', 'KO', 'NKE', 'DIS',
    'JPM', 'BAC', 'V', 'MA', 'WMT'
]  # 25 major companies

start_date = '2018-01-01'
end_date = '2025-01-01'
model_dir = "alphawave_model"

os.makedirs(model_dir, exist_ok=True)

# ---------------------------
# 2. Download and Combine Data
# ---------------------------
print("Fetching data for all stocks...")
data = yf.download(stock_symbols, start=start_date, end=end_date)['Close']
data = data.dropna(axis=1)  # remove columns with missing values

print(f"Final number of stocks used: {len(data.columns)}")
print("Stocks:", list(data.columns))

# Scale the entire dataframe
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)
scaled_df = pd.DataFrame(scaled_data, columns=data.columns, index=data.index)

# ---------------------------
# 3. Prepare Training and Testing Data
# ---------------------------
training_data_len = int(len(scaled_df) * 0.8)
train_data = scaled_df.iloc[:training_data_len]
test_data = scaled_df.iloc[training_data_len - 60:]  # overlap for sequence

def create_multifeature_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(time_step, len(dataset)):
        X.append(dataset.iloc[i - time_step:i].values)
        y.append(dataset.iloc[i].values)
    return np.array(X), np.array(y)

X_train, y_train = create_multifeature_dataset(train_data)
X_test, y_test = create_multifeature_dataset(test_data)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

# ---------------------------
# 4. Build the Universal LSTM Model
# ---------------------------
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(128, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(y_train.shape[1])  # predict all stocks
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# ---------------------------
# 5. Train the Model
# ---------------------------
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_split=0.1, verbose=1)

# ---------------------------
# 6. Save Model and Scaler
# ---------------------------
model_path_h5 = os.path.join(model_dir, "alphawave_universal_model.h5")
model_path_keras = os.path.join(model_dir, "alphawave_universal_model.keras")
scaler_path = os.path.join(model_dir, "alphawave_scaler.pkl")

model.save(model_path_h5)
model.save(model_path_keras)
joblib.dump(scaler, scaler_path)

print(f"\n✅ Model saved successfully to:\n - {model_path_h5}\n - {model_path_keras}")
print(f"✅ Scaler saved to: {scaler_path}")

# ---------------------------
# 7. Evaluate and Plot Sample Stocks
# ---------------------------
predictions = model.predict(X_test)
predictions_rescaled = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

rmse_per_stock = {}
for idx, stock in enumerate(data.columns):
    rmse = math.sqrt(mean_squared_error(y_test_actual[:, idx], predictions_rescaled[:, idx]))
    rmse_per_stock[stock] = rmse

print("\nRMSE per Stock:")
for stock, score in rmse_per_stock.items():
    print(f"{stock}: {score:.2f}")

sample_stocks = ['AAPL', 'TSLA', 'GOOG', 'AMZN']
for stock in sample_stocks:
    idx = list(data.columns).index(stock)
    plt.figure(figsize=(10, 5))
    plt.plot(data.index[training_data_len:], y_test_actual[:, idx], label='Actual', color='blue')
    plt.plot(data.index[training_data_len:], predictions_rescaled[:, idx], label='Predicted', color='red')
    plt.title(f'{stock} Price Prediction (AlphaWave Universal LSTM)')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.show()

# ---------------------------
# 8. Predict Next 30 Days for All Stocks
# ---------------------------
last_60_days = scaled_df.iloc[-60:].values.reshape(1, 60, len(data.columns))
future_predictions = []

for _ in range(30):
    next_pred = model.predict(last_60_days)[0]
    future_predictions.append(next_pred)
    next_input = np.append(last_60_days[:, 1:, :], [[next_pred]], axis=1)
    last_60_days = next_input

future_predictions = scaler.inverse_transform(np.array(future_predictions))

for stock in sample_stocks:
    idx = list(data.columns).index(stock)
    plt.figure(figsize=(8, 4))
    plt.plot(range(1, 31), future_predictions[:, idx], marker='o', color='green')
    plt.title(f'{stock} - 30-Day Forecast (AlphaWave Universal LSTM)')
    plt.xlabel('Days Ahead')
    plt.ylabel('Predicted Price (USD)')
    plt.grid()
    plt.show()


In [ ]:
# AlphaWave v3 – Optimized Universal LSTM (No model saving)
# Author: AriseAK

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import math

# ---------------------------
# 1. Configuration
# ---------------------------
stock_symbols = [
    'AAPL', 'MSFT', 'TSLA', 'GOOG', 'AMZN', 'NVDA', 'META', 'NFLX', 'AMD', 'INTC',
    'ADBE', 'ORCL', 'IBM', 'PYPL', 'CRM', 'CSCO', 'PEP', 'KO', 'NKE', 'DIS',
    'JPM', 'BAC', 'V', 'MA', 'WMT'
]
start_date = '2018-01-01'
end_date = '2025-01-01'
lookback = 90

# ---------------------------
# 2. Helper Functions
# ---------------------------

def compute_rsi(series, period=14):
    delta = series.diff(1)
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)
    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()
    rs = avg_gain / (avg_loss + 1e-10)
    return 100 - (100 / (1 + rs))

def create_multifeature_dataset(df, time_step=60):
    X, y = [], []
    for i in range(time_step, len(df)):
        X.append(df.iloc[i - time_step:i].values)
        y.append(df.iloc[i].values)
    return np.array(X), np.array(y)

# ---------------------------
# 3. Data Loading & Feature Engineering
# ---------------------------
print("Fetching stock data...")
data = yf.download(stock_symbols, start=start_date, end=end_date)['Close']
data = data.dropna(axis=1)

# Add technical indicators for each stock
print("Adding technical indicators...")
features = pd.DataFrame(index=data.index)
for col in data.columns:
    df = pd.DataFrame()
    df[f'{col}_Close'] = data[col]
    df[f'{col}_MA7'] = data[col].rolling(window=7).mean()
    df[f'{col}_MA21'] = data[col].rolling(window=21).mean()
    df[f'{col}_RSI'] = compute_rsi(data[col])
    df[f'{col}_Volatility'] = data[col].rolling(window=7).std()
    df = df.fillna(method='bfill').fillna(method='ffill')

    # Per-stock MinMax scaling (0–1)
    scaler = MinMaxScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)
    features = pd.concat([features, df_scaled], axis=1)

print(f"Final feature count: {features.shape[1]} (per stock indicators included)")

# ---------------------------
# 4. Train-Test Split & Dataset Creation
# ---------------------------
training_len = int(len(features) * 0.8)
train_data = features.iloc[:training_len]
test_data = features.iloc[training_len - lookback:]  # overlap

X_train, y_train = create_multifeature_dataset(train_data, time_step=lookback)
X_test, y_test = create_multifeature_dataset(test_data, time_step=lookback)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

# ---------------------------
# 5. Build Optimized LSTM Model
# ---------------------------
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    BatchNormalization(),
    Dropout(0.2),

    LSTM(128, return_sequences=True),
    BatchNormalization(),
    Dropout(0.2),

    LSTM(64, return_sequences=False),
    Dropout(0.2),

    Dense(64, activation='relu'),
    Dense(y_train.shape[1])
])

optimizer = Adam(learning_rate=0.0003)
model.compile(optimizer=optimizer, loss='mean_squared_error')
model.summary()

# ---------------------------
# 6. Train with EarlyStopping
# ---------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    epochs=60,
    batch_size=64,
    validation_split=0.1,
    shuffle=False,
    callbacks=[early_stop],
    verbose=1
)

# ---------------------------
# 7. Evaluate Performance
# ---------------------------
predictions = model.predict(X_test)
y_test_actual = y_test  # Already scaled per-stock
rmse = math.sqrt(mean_squared_error(y_test.flatten(), predictions.flatten()))
print(f"\nOverall RMSE (all stocks/features): {rmse:.4f}")

# ---------------------------
# 8. Visualize Key Stocks
# ---------------------------
# Extract key columns to visualize (close price columns only)
close_cols = [col for col in features.columns if '_Close' in col]
num_features_per_stock = len(features.columns) // len(close_cols)

for stock in ['AAPL', 'TSLA', 'GOOG', 'AMZN']:
    col = f'{stock}_Close'
    idx = list(features.columns).index(col)

    actual = y_test[:, idx]
    predicted = predictions[:, idx]

    plt.figure(figsize=(10, 5))
    plt.plot(features.index[training_len:], actual, label='Actual', color='blue')
    plt.plot(features.index[training_len:], predicted, label='Predicted', color='red')
    plt.title(f'{stock} Price Prediction (AlphaWave v3 Optimized Universal LSTM)')
    plt.xlabel('Date')
    plt.ylabel('Normalized Price')
    plt.legend()
    plt.show()

# ---------------------------
# 9. Predict Next 30 Days for All Stocks
# ---------------------------
last_seq = features.iloc[-lookback:].values.reshape(1, lookback, features.shape[1])
future_preds = []

for _ in range(30):
    pred = model.predict(last_seq)[0]
    future_preds.append(pred)
    last_seq = np.append(last_seq[:, 1:, :], [[pred]], axis=1)

future_preds = np.array(future_preds)

# Plot 30-day forecasts (normalized values)
for stock in ['AAPL', 'TSLA', 'GOOG', 'AMZN']:
    col = f'{stock}_Close'
    idx = list(features.columns).index(col)
    plt.figure(figsize=(8, 4))
    plt.plot(range(1, 31), future_preds[:, idx], color='green', marker='o')
    plt.title(f'{stock} - 30-Day Forecast (AlphaWave v3 Optimized LSTM)')
    plt.xlabel('Days Ahead')
    plt.ylabel('Normalized Predicted Price')
    plt.grid()
    plt.show()

print("\n✅ Training complete. Model not saved (as requested).")
print("You can later save it anytime using:")
print("    model.save('alphawave_model_v3.h5')")


In [ ]:
# AlphaWave v4 – Hybrid Universal LSTM (Stock-Aware, Volume, Market Index)
# Author: AriseAK

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Concatenate, Embedding, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import math

# ---------------------------
# 1. Configuration
# ---------------------------
stock_symbols = [
    'AAPL', 'MSFT', 'TSLA', 'GOOG', 'AMZN', 'NVDA', 'META', 'NFLX', 'AMD', 'INTC',
    'ADBE', 'ORCL', 'IBM', 'PYPL', 'CRM', 'CSCO', 'PEP', 'KO', 'NKE', 'DIS',
    'JPM', 'BAC', 'V', 'MA', 'WMT'
]
start_date = '2015-01-01'
end_date = '2025-01-01'
lookback = 120

# ---------------------------
# 2. Load Stock & Market Data
# ---------------------------
print("Fetching data for all stocks...")
data = yf.download(stock_symbols, start=start_date, end=end_date)
close_df = data['Close']
volume_df = data['Volume']

# Drop missing columns if any stock has incomplete data
close_df = close_df.dropna(axis=1)
volume_df = volume_df[close_df.columns]

# Add SPY (S&P500) as a global index feature
print("Adding market index (SPY)...")
spy = yf.download('SPY', start=start_date, end=end_date)['Close']
spy = spy.reindex(close_df.index).fillna(method='ffill')

# ---------------------------
# 3. Feature Engineering
# ---------------------------
# Combine close, volume, and SPY
features = pd.DataFrame(index=close_df.index)
for stock in close_df.columns:
    features[f'{stock}_Close'] = close_df[stock]
    features[f'{stock}_Volume'] = volume_df[stock]

features['SPY_Close'] = spy

# Fill missing
features = features.fillna(method='ffill').fillna(method='bfill')

# Standard scaling (better than MinMax for financial data)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(features)
scaled_df = pd.DataFrame(scaled_data, columns=features.columns, index=features.index)

print(f"Final feature matrix shape: {scaled_df.shape}")

# ---------------------------
# 4. Create Sequences
# ---------------------------
def create_dataset(dataset, stocks, time_step=60):
    X, y, stock_ids = [], [], []
    for stock_idx, stock in enumerate(stocks):
        close_col = f'{stock}_Close'
        vol_col = f'{stock}_Volume'

        stock_features = dataset[[close_col, vol_col, 'SPY_Close']].values  # 3 features per time step
        for i in range(time_step, len(stock_features)):
            X.append(stock_features[i - time_step:i])
            y.append(stock_features[i, 0])  # predict closing price
            stock_ids.append(stock_idx)  # one-hot or embedding index
    return np.array(X), np.array(y), np.array(stock_ids)

print("Creating training sequences...")
training_len = int(len(scaled_df) * 0.8)
train_data = scaled_df.iloc[:training_len]
test_data = scaled_df.iloc[training_len - lookback:]

X_train, y_train, stock_train_ids = create_dataset(train_data, close_df.columns, lookback)
X_test, y_test, stock_test_ids = create_dataset(test_data, close_df.columns, lookback)

print("X_train:", X_train.shape, "X_test:", X_test.shape)

# ---------------------------
# 5. Build Stock-Aware Hybrid LSTM
# ---------------------------
# Inputs
price_input = Input(shape=(X_train.shape[1], X_train.shape[2]), name='price_input')
stock_input = Input(shape=(1,), name='stock_input')

# Stock Embedding (gives the model stock identity awareness)
embedding = Embedding(input_dim=len(close_df.columns), output_dim=5, input_length=1)(stock_input)
embedding = Flatten()(embedding)

# LSTM feature extractor
x = LSTM(128, return_sequences=True)(price_input)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = LSTM(64, return_sequences=False)(x)
x = Dropout(0.2)(x)

# Combine LSTM output + stock identity
combined = Concatenate()([x, embedding])
combined = Dense(64, activation='relu')(combined)
combined = Dropout(0.2)(combined)
output = Dense(1)(combined)

model = Model(inputs=[price_input, stock_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0003), loss='mean_squared_error')
model.summary()

# ---------------------------
# 6. Train Model
# ---------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    [X_train, stock_train_ids],
    y_train,
    epochs=80,
    batch_size=64,
    validation_split=0.1,
    shuffle=False,
    callbacks=[early_stop],
    verbose=1
)

# ---------------------------
# 7. Evaluate & Predict
# ---------------------------
predictions = model.predict([X_test, stock_test_ids])
rmse = math.sqrt(mean_squared_error(y_test, predictions))
print(f"\nOverall RMSE: {rmse:.4f}")

# ---------------------------
# 8. Plot Predictions for Sample Stocks
# ---------------------------
from sklearn.metrics import r2_score
unique_stocks = close_df.columns[:4]  # visualize a few

for stock in unique_stocks:
    idx = np.where(stock_test_ids == list(close_df.columns).index(stock))[0]
    plt.figure(figsize=(10,5))
    plt.plot(y_test[idx], label='Actual', color='blue')
    plt.plot(predictions[idx], label='Predicted', color='red')
    plt.title(f"{stock} Price Prediction (AlphaWave v4 Hybrid Universal LSTM)")
    plt.xlabel("Days Ahead")
    plt.ylabel("Standardized Price")
    plt.legend()
    plt.show()

# ---------------------------
# 9. Optional: 30-Day Forecast
# ---------------------------
# For a single stock, e.g., AAPL
stock_idx = list(close_df.columns).index('AAPL')
last_seq = scaled_df[[f'AAPL_Close', f'AAPL_Volume', 'SPY_Close']].iloc[-lookback:].values.reshape(1, lookback, 3)
stock_input = np.array([[stock_idx]])
future_preds = []


last_seq = np.array(last_seq, dtype=np.float32)
stock_input = np.array([[stock_idx]], dtype=np.int32)


for _ in range(30):
    pred = model.predict([last_seq, stock_input], verbose=0)[0, 0]
    future_preds.append(pred)
    new_frame = np.array([[[pred, last_seq[0, -1, 1], last_seq[0, -1, 2]]]], dtype=np.float32)
    last_seq = np.append(last_seq[:, 1:, :], new_frame, axis=1)


plt.figure(figsize=(8,4))
plt.plot(range(1,31), future_preds, color='green', marker='o')
plt.title("AAPL - 30-Day Forecast (AlphaWave v4 Hybrid LSTM)")
plt.xlabel("Days Ahead")
plt.ylabel("Standardized Predicted Price")
plt.grid()
plt.show()

print("\n✅ Training complete — AlphaWave v4 is now stock-aware and market-contextualized.")


In [ ]:
# AlphaWave v4.1 – Hybrid LSTM (10 Stocks, Per-Stock Scaling)
# Author: AriseAK

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Concatenate, Embedding, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import math

# ---------------------------
# 1. Config
# ---------------------------
stock_symbols = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'NVDA', 'META', 'TSLA', 'AMD', 'ADBE', 'NFLX']
start_date = '2015-01-01'
end_date = '2025-01-01'
lookback = 90  # shorter window = more responsive

# ---------------------------
# 2. Load Stock & Market Data
# ---------------------------
print("Fetching stock data...")
data = yf.download(stock_symbols, start=start_date, end=end_date)
close_df = data['Close']
volume_df = data['Volume']

# Drop incomplete stocks
close_df = close_df.dropna(axis=1)
volume_df = volume_df[close_df.columns]

# Add SPY as market context
print("Adding SPY (S&P500 Index)...")
spy = yf.download('SPY', start=start_date, end=end_date)['Close']
spy = spy.reindex(close_df.index).fillna(method='ffill')

# ---------------------------
# 3. Per-Stock Standard Scaling
# ---------------------------
scalers = {}
scaled_close = pd.DataFrame(index=close_df.index)
scaled_volume = pd.DataFrame(index=close_df.index)

for stock in close_df.columns:
    scaler = StandardScaler()
    scaled_close[stock] = scaler.fit_transform(close_df[stock].values.reshape(-1, 1)).flatten()
    scaled_volume[stock] = StandardScaler().fit_transform(volume_df[stock].values.reshape(-1, 1)).flatten()
    scalers[stock] = scaler

spy_scaler = StandardScaler()
scaled_spy = spy_scaler.fit_transform(spy.values.reshape(-1, 1)).flatten()

# ---------------------------
# 4. Combine All Features
# ---------------------------
scaled_features = pd.DataFrame(index=close_df.index)
for stock in close_df.columns:
    scaled_features[f'{stock}_Close'] = scaled_close[stock]
    scaled_features[f'{stock}_Volume'] = scaled_volume[stock]
scaled_features['SPY_Close'] = scaled_spy

print("Final feature matrix shape:", scaled_features.shape)

# ---------------------------
# 5. Create Sequences
# ---------------------------
def create_dataset(dataset, stocks, time_step=60):
    X, y, stock_ids = [], [], []
    for stock_idx, stock in enumerate(stocks):
        close_col = f'{stock}_Close'
        vol_col = f'{stock}_Volume'
        stock_features = dataset[[close_col, vol_col, 'SPY_Close']].values
        
        for i in range(time_step, len(stock_features)):
            X.append(stock_features[i - time_step:i])
            y.append(stock_features[i, 0])  # predict Close
            stock_ids.append(stock_idx)
    return np.array(X), np.array(y), np.array(stock_ids)

train_len = int(len(scaled_features) * 0.8)
train_data = scaled_features.iloc[:train_len]
test_data = scaled_features.iloc[train_len - lookback:]

X_train, y_train, stock_train_ids = create_dataset(train_data, close_df.columns, lookback)
X_test, y_test, stock_test_ids = create_dataset(test_data, close_df.columns, lookback)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

# ---------------------------
# 6. Build Hybrid Model
# ---------------------------
price_input = Input(shape=(X_train.shape[1], X_train.shape[2]), name='price_input')
stock_input = Input(shape=(1,), name='stock_input')

# Stock embedding
embedding = Embedding(input_dim=len(close_df.columns), output_dim=5, input_length=1)(stock_input)
embedding = Flatten()(embedding)

# LSTM feature extractor
x = LSTM(128, return_sequences=True)(price_input)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

x = LSTM(64, return_sequences=False)(x)
x = Dropout(0.1)(x)

# Merge LSTM + embedding
combined = Concatenate()([x, embedding])
combined = Dense(64, activation='relu')(combined)
combined = Dropout(0.1)(combined)
output = Dense(1)(combined)

model = Model(inputs=[price_input, stock_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0003), loss='mean_squared_error')
model.summary()

# ---------------------------
# 7. Train Model
# ---------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    [X_train, stock_train_ids],
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    shuffle=False,
    callbacks=[early_stop],
    verbose=1
)

# ---------------------------
# 8. Evaluate
# ---------------------------
predictions = model.predict([X_test, stock_test_ids])
rmse = math.sqrt(mean_squared_error(y_test, predictions))
print(f"Overall RMSE: {rmse:.4f}")

# ---------------------------
# 9. Visualize Sample Stocks
# ---------------------------
unique_stocks = close_df.columns[:4]  # visualize 4

for stock in unique_stocks:
    idx = np.where(stock_test_ids == list(close_df.columns).index(stock))[0]
    plt.figure(figsize=(10,5))
    plt.plot(y_test[idx], label='Actual', color='blue')
    plt.plot(predictions[idx], label='Predicted', color='red')
    plt.title(f"{stock} Price Prediction (AlphaWave v4.1 Hybrid LSTM)")
    plt.xlabel("Days Ahead")
    plt.ylabel("Standardized Price")
    plt.legend()
    plt.show()

# ---------------------------
# 10. 30-Day Forecast Example
# ---------------------------
print("\nGenerating 30-day forecast for AAPL...")

stock_idx = list(close_df.columns).index('AAPL')
last_seq = scaled_features[[f'AAPL_Close', f'AAPL_Volume', 'SPY_Close']].iloc[-lookback:].values
last_seq = np.array(last_seq, dtype=np.float32).reshape(1, lookback, 3)
stock_input = np.array([[stock_idx]], dtype=np.int32)

future_preds = []

for _ in range(30):
    pred = model.predict([last_seq, stock_input], verbose=0)[0, 0]
    future_preds.append(pred)
    new_frame = np.array([[[pred, last_seq[0, -1, 1], last_seq[0, -1, 2]]]], dtype=np.float32)
    last_seq = np.append(last_seq[:, 1:, :], new_frame, axis=1)

plt.figure(figsize=(8,4))
plt.plot(range(1,31), future_preds, color='green', marker='o')
plt.title("AAPL - 30-Day Forecast (AlphaWave v4.1)")
plt.xlabel("Days Ahead")
plt.ylabel("Standardized Predicted Price")
plt.grid()
plt.show()


In [ ]:
# AlphaWave vX – Unified Model with Stock Embeddings
# Author: AriseAK

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Embedding, Flatten, Concatenate
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import math

# ---------------------------
# 1. Config
# ---------------------------
stock_symbols = [
    'AAPL', 'MSFT', 'AMZN', 'GOOG', 'NVDA', 'META', 'TSLA', 
    'AMD', 'ADBE', 'NFLX', 'INTC', 'IBM', 'PYPL', 'ORCL', 
    'CSCO', 'CRM', 'QCOM', 'AVGO', 'TXN', 'SHOP', 'UBER', 'SNOW', 
    'BABA', 'PLTR', 'COIN'
]
start_date = '2015-01-01'
end_date = '2025-01-01'
lookback = 90  # sequence window

# ---------------------------
# 2. Load Stock & Market Data
# ---------------------------
print("Fetching stock data...")
data = yf.download(stock_symbols, start=start_date, end=end_date)
close_df = data['Close']
volume_df = data['Volume']
close_df = close_df.dropna(axis=1)
volume_df = volume_df[close_df.columns]

print("Adding SPY (S&P500 Index)...")
spy = yf.download('SPY', start=start_date, end=end_date)['Close']
spy = spy.reindex(close_df.index).fillna(method='ffill')

# ---------------------------
# 3. Per-Stock Scaling
# ---------------------------
scalers = {}
scaled_close, scaled_volume = pd.DataFrame(index=close_df.index), pd.DataFrame(index=close_df.index)

for stock in close_df.columns:
    scaler = StandardScaler()
    scaled_close[stock] = scaler.fit_transform(close_df[stock].values.reshape(-1, 1)).flatten()
    scaled_volume[stock] = StandardScaler().fit_transform(volume_df[stock].values.reshape(-1, 1)).flatten()
    scalers[stock] = scaler

spy_scaler = StandardScaler()
scaled_spy = spy_scaler.fit_transform(spy.values.reshape(-1, 1)).flatten()

# ---------------------------
# 4. Create Dataset
# ---------------------------
def create_dataset(dataset, stocks, time_step=60):
    X, y, stock_ids = [], [], []
    for stock_idx, stock in enumerate(stocks):
        close_col = f'{stock}_Close'
        vol_col = f'{stock}_Volume'
        stock_features = dataset[[close_col, vol_col, 'SPY_Close']].values
        
        for i in range(time_step, len(stock_features)):
            X.append(stock_features[i - time_step:i])
            y.append(stock_features[i, 0])
            stock_ids.append(stock_idx)
    return np.array(X), np.array(y), np.array(stock_ids)

scaled_features = pd.DataFrame(index=close_df.index)
for stock in close_df.columns:
    scaled_features[f'{stock}_Close'] = scaled_close[stock]
    scaled_features[f'{stock}_Volume'] = scaled_volume[stock]
scaled_features['SPY_Close'] = scaled_spy

train_len = int(len(scaled_features) * 0.8)
train_data = scaled_features.iloc[:train_len]
test_data = scaled_features.iloc[train_len - lookback:]

X_train, y_train, stock_train_ids = create_dataset(train_data, close_df.columns, lookback)
X_test, y_test, stock_test_ids = create_dataset(test_data, close_df.columns, lookback)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

# ---------------------------
# 5. Build Unified Model
# ---------------------------
price_input = Input(shape=(X_train.shape[1], X_train.shape[2]), name='price_input')
stock_input = Input(shape=(1,), name='stock_input')

# Stock embedding layer – each stock gets an 8D learned vector
embedding = Embedding(input_dim=len(close_df.columns), output_dim=8, input_length=1, name='stock_embedding')(stock_input)
embedding = Flatten()(embedding)

# Shared LSTM backbone
x = LSTM(128, return_sequences=True)(price_input)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

x = LSTM(64, return_sequences=False)(x)
x = Dropout(0.1)(x)

# Merge temporal features with stock personality
combined = Concatenate()([x, embedding])
combined = Dense(64, activation='relu')(combined)
combined = Dropout(0.1)(combined)
output = Dense(1)(combined)

model = Model(inputs=[price_input, stock_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0003), loss='mean_squared_error')
model.summary()

# ---------------------------
# 6. Train
# ---------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    [X_train, stock_train_ids],
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    shuffle=True,
    callbacks=[early_stop],
    verbose=1
)

# ---------------------------
# 7. Evaluate
# ---------------------------
predictions = model.predict([X_test, stock_test_ids])
rmse = math.sqrt(mean_squared_error(y_test, predictions))
print(f"Overall RMSE: {rmse:.4f}")

# ---------------------------
# 8. Visualize Few Stocks
# ---------------------------
unique_stocks = close_df.columns[:4]

for stock in unique_stocks:
    idx = np.where(stock_test_ids == list(close_df.columns).index(stock))[0]
    plt.figure(figsize=(10,5))
    plt.plot(y_test[idx], label='Actual', color='blue')
    plt.plot(predictions[idx], label='Predicted', color='red')
    plt.title(f"{stock} Stock Prediction (AlphaWave vX Unified Model)")
    plt.xlabel("Days")
    plt.ylabel("Standardized Price")
    plt.legend()
    plt.show()

# ---------------------------
# 9. Extract and Save Stock Embeddings
# ---------------------------
embedding_weights = model.get_layer('stock_embedding').get_weights()[0]
stock_embeddings = pd.DataFrame(embedding_weights, index=close_df.columns)
print("\nLearned Stock Embeddings (first 5):")
print(stock_embeddings.head())

# ---------------------------
# 10. Compute Stock Similarities
# ---------------------------
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix = cosine_similarity(stock_embeddings.values)
sim_df = pd.DataFrame(sim_matrix, index=close_df.columns, columns=close_df.columns)

print("\nTop 5 Most Similar Stocks:")
for stock in close_df.columns[:5]:
    similar = sim_df[stock].sort_values(ascending=False)[1:4]
    print(f"{stock}: {', '.join(similar.index)}")


In [ ]:
# ==============================================
# AlphaWave vX.3 - Simplified, Regularized, Robust LSTM
# Handles overfitting + supports multiple stocks
# ==============================================

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout, GaussianNoise
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

# ------------------------------------------------------------
# 1️⃣ Load Stock Data (10 stocks)
# ------------------------------------------------------------
tickers = [
    'AAPL', 'MSFT', 'AMZN', 'GOOG', 'NVDA', 'META', 'TSLA', 'AMD', 'ADBE', 'NFLX',
    'INTC', 'IBM', 'PYPL', 'ORCL', 'CSCO', 'CRM', 'QCOM', 'AVGO', 'TXN', 'SHOP',
    'UBER', 'SNOW', 'BABA', 'PLTR', 'COIN'
]


data_dict = {}
for ticker in tickers:
    print(f"Downloading data for {ticker}...")
    data_dict[ticker] = yf.download(ticker, start='2015-01-01', end='2025-01-01')

# ------------------------------------------------------------
# 2️⃣ Preprocess Data (Close + Volume)
# ------------------------------------------------------------
def preprocess_data(df):
    df = df[['Close', 'Volume']].dropna()
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df)
    return scaled, scaler

scaled_data = {}
scalers = {}

for t in tickers:
    scaled_data[t], scalers[t] = preprocess_data(data_dict[t])

# ------------------------------------------------------------
# 3️⃣ Create Sequential Dataset (Lookback = 90 days)
# ------------------------------------------------------------
def create_sequences(data, seq_len=90):
    X, y = [], []
    for i in range(seq_len, len(data)):
        X.append(data[i - seq_len:i])
        y.append(data[i, 0])  # Predict 'Close'
    return np.array(X), np.array(y)

seq_len = 90
X_all, y_all = [], []

for t in tickers:
    X, y = create_sequences(scaled_data[t], seq_len)
    X_all.append(X)
    y_all.append(y)

X_all = np.concatenate(X_all)
y_all = np.concatenate(y_all)

print(f"\nCombined dataset shape: {X_all.shape}, {y_all.shape}")

# ------------------------------------------------------------
# 4️⃣ Time-based Split (Train 70% / Val 15% / Test 15%)
# ------------------------------------------------------------
train_size = int(len(X_all) * 0.7)
val_size = int(len(X_all) * 0.15)

X_train = X_all[:train_size]
y_train = y_all[:train_size]
X_val = X_all[train_size:train_size + val_size]
y_val = y_all[train_size:train_size + val_size]
X_test = X_all[train_size + val_size:]
y_test = y_all[train_size + val_size:]

print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")

# ------------------------------------------------------------
# 5️⃣ Build Simplified & Regularized LSTM Model
# ------------------------------------------------------------
input_layer = Input(shape=(seq_len, 2))
x = GaussianNoise(0.01)(input_layer)  # Stabilize inputs

# ✅ Single LSTM layer (simplified)
x = LSTM(64, dropout=0.2, kernel_regularizer=l2(0.001))(x)


# ✅ Light dense head
x = Dense(32, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.3)(x)

output_layer = Dense(1)(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mse')

model.summary()

# ------------------------------------------------------------
# 6️⃣ Train with EarlyStopping (Prevent Overfitting)
# ------------------------------------------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1
)

# ------------------------------------------------------------
# 7️⃣ Evaluate Model (RMSE)
# ------------------------------------------------------------
train_pred = model.predict(X_train)
val_pred = model.predict(X_val)
test_pred = model.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))

print("\n========== RMSE Summary ==========")
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Val RMSE:   {val_rmse:.4f}")
print(f"Test RMSE:  {test_rmse:.4f}")
print("==================================\n")

# ------------------------------------------------------------
# 8️⃣ Training vs Validation Loss Visualization
# ------------------------------------------------------------
plt.figure(figsize=(10,5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
plt.title("Training vs Validation Loss (AlphaWave vX.3)")
plt.xlabel("Epochs")
plt.ylabel("MSE Loss")
plt.legend()
plt.grid(True)
plt.show()

# ------------------------------------------------------------
# 9️⃣ Evaluate Per Stock + Visualize Predictions
# ------------------------------------------------------------
results = []

for t in tickers:
    X_stock, y_stock = create_sequences(scaled_data[t], seq_len)
    pred_stock = model.predict(X_stock)

    rmse = np.sqrt(mean_squared_error(y_stock, pred_stock))
    results.append({'Stock': t, 'RMSE': rmse})

    plt.figure(figsize=(10,5))
    plt.plot(y_stock[-500:], label='Actual', color='blue')
    plt.plot(pred_stock[-500:], label='Predicted', color='red')
    plt.title(f"{t} Stock Prediction (AlphaWave vX.3)")
    plt.xlabel("Days")
    plt.ylabel("Standardized Price")
    plt.legend()
    plt.grid(True)
    plt.show()

results_df = pd.DataFrame(results)
print("\nPer-stock RMSE summary:\n", results_df)

# ------------------------------------------------------------
# 🔟 (Optional) Save Model + Metrics
# ------------------------------------------------------------
# model.save("alphawave_vX3_optimized_lstm.h5")
# results_df.to_csv("alphawave_vX3_rmse_summary.csv", index=False)


In [ ]:
# ------------------------------------------------------------
# Save trained AlphaWave model + metrics (post-training)
# ------------------------------------------------------------
from datetime import datetime
import pandas as pd

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
model_name = f"alphawave_vX3_trained_{timestamp}.keras"
metrics_name = f"alphawave_vX3_metrics_{timestamp}.csv"

# Save the model in Keras format (recommended)
model.save(model_name)
print(f"✅ Model saved successfully as '{model_name}'")

# Optional: if RMSE variables exist, save them too
try:
    metrics_df = pd.DataFrame({
        'Metric': ['Train_RMSE', 'Val_RMSE', 'Test_RMSE'],
        'Value': [train_rmse, val_rmse, test_rmse]
    })
    metrics_df.to_csv(metrics_name, index=False)
    print(f"📊 Metrics saved as '{metrics_name}'")
except NameError:
    print("⚠️ RMSE metrics not found — skipping metrics save.")



In [ ]:
# ===========================================================
# Save all stock scalers for AlphaWave vX.3
# ===========================================================

import joblib
import os

# Create a dedicated folder for scalers
os.makedirs("alphawave_scalers", exist_ok=True)

# Loop through each stock and save its StandardScaler
for t in tickers:
    file_path = os.path.join("alphawave_scalers", f"scaler_{t}.pkl")
    joblib.dump(scalers[t], file_path)

print(f"✅ All {len(tickers)} stock scalers saved successfully in the 'alphawave_scalers/' folder.")


In [ ]:
# ===========================================================
# AlphaWave vX.3 Generalization & Future Data Evaluation
# ===========================================================

from datetime import datetime
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import yfinance as yf

# -----------------------------------------------------------
# 1️⃣ Train vs Validation RMSE Ratio (Overfitting Metric)
# -----------------------------------------------------------

print("========== Train vs Validation RMSE Ratio ==========")

try:
    train_rmse_ratio = train_rmse / val_rmse
    print(f"Train RMSE:       {train_rmse:.4f}")
    print(f"Validation RMSE:  {val_rmse:.4f}")
    print(f"RMSE Ratio:       {train_rmse_ratio:.3f}")

    if train_rmse_ratio < 1.1:
        print("✅ Excellent generalization — model not overfitting.")
    elif 1.1 <= train_rmse_ratio <= 1.3:
        print("⚖️  Mild overfitting, acceptable range.")
    else:
        print("⚠️  Possible overfitting — model fits training data too tightly.")
except NameError:
    print("⚠️ RMSE variables not found. Skipping this part.")
    train_rmse_ratio = None

# -----------------------------------------------------------
# 2️⃣ Evaluate on Unseen 2025 Data (Future Drift Test)
# -----------------------------------------------------------

print("\n========== Future Data Evaluation (2025 onward) ==========")

future_results = []

for t in tickers:
    print(f"\nEvaluating {t}...")
    try:
        # Download unseen data
        future_data = yf.download(t, start='2025-01-01', end=datetime.today().strftime('%Y-%m-%d'))
        if len(future_data) < 60:
            print(f"⚠️  Not enough 2025 data for {t}, skipping.")
            continue

        # Preprocess using saved scaler
        scaled_future = scalers[t].transform(future_data[['Close', 'Volume']])
        
        # Create sequences
        X_future, y_future = [], []
        for i in range(seq_len, len(scaled_future)):
            X_future.append(scaled_future[i - seq_len:i])
            y_future.append(scaled_future[i, 0])
        X_future, y_future = np.array(X_future), np.array(y_future)

        # Predict & evaluate
        future_pred = model.predict(X_future, verbose=0)
        future_rmse = np.sqrt(mean_squared_error(y_future, future_pred))
        future_results.append({'Stock': t, 'Future_RMSE': future_rmse})

    except Exception as e:
        print(f"❌ Error evaluating {t}: {e}")

# Create summary DataFrame
future_df = pd.DataFrame(future_results)
print("\n========== Future RMSE Summary ==========")
print(future_df)

# -----------------------------------------------------------
# 3️⃣ Compare test vs future RMSE (Model Drift)
# -----------------------------------------------------------

if not future_df.empty:
    try:
        avg_future_rmse = future_df['Future_RMSE'].mean()
        print("\n========== Model Drift Summary ==========")
        print(f"Average Test RMSE (from training):  {test_rmse:.4f}")
        print(f"Average Future RMSE (2025 unseen):  {avg_future_rmse:.4f}")
        drift_ratio = avg_future_rmse / test_rmse
        print(f"RMSE Drift Ratio: {drift_ratio:.3f}")

        if drift_ratio < 1.2:
            print("✅ Model generalizes stably across unseen 2025 data.")
        elif 1.2 <= drift_ratio <= 1.5:
            print("⚖️  Moderate drift — retraining in 6 months recommended.")
        else:
            print("⚠️  Significant drift — consider retraining with recent data.")
    except NameError:
        print("⚠️ Test RMSE not found. Only showing future RMSEs.")
else:
    print("⚠️ No valid results for future data evaluation.")


In [ ]:
# ===========================================================
# AlphaWave vX.4 — Retraining with 2025 Data (Full Pipeline)
# ===========================================================

import yfinance as yf
import numpy as np
import pandas as pd
import joblib
import os
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# -----------------------------------------------------------
# 1️⃣ Setup — Stock List and Configs
# -----------------------------------------------------------
tickers = [
    'AAPL', 'MSFT', 'AMZN', 'GOOG', 'NVDA', 'META', 'TSLA', 'AMD', 'ADBE', 'NFLX',
    'INTC', 'IBM', 'PYPL', 'ORCL', 'CSCO', 'CRM', 'QCOM', 'AVGO', 'TXN', 'SHOP',
    'UBER', 'SNOW', 'BABA', 'PLTR', 'COIN'
]

seq_len = 60  # Lookback window
start_date = '2015-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

os.makedirs("alphawave_vX4_checkpoints", exist_ok=True)
os.makedirs("alphawave_vX4_scalers", exist_ok=True)

# -----------------------------------------------------------
# 2️⃣ Data Loading and Preprocessing
# -----------------------------------------------------------
def preprocess_data(df):
    df = df[['Close', 'Volume']].dropna()
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df)
    return scaled, scaler

data_dict, scaled_data, scalers = {}, {}, {}

print("📈 Fetching and preprocessing updated stock data...\n")
for t in tickers:
    df = yf.download(t, start=start_date, end=end_date)
    if len(df) > 0:
        data_dict[t] = df
        scaled_data[t], scalers[t] = preprocess_data(df)
    else:
        print(f"⚠️ Skipping {t}: no data available.")

# Save updated scalers
for t in tickers:
    joblib.dump(scalers[t], f"alphawave_vX4_scalers/scaler_{t}.pkl")

print("\n✅ All scalers updated and saved successfully.")

# -----------------------------------------------------------
# 3️⃣ Create Training Dataset
# -----------------------------------------------------------
def create_dataset(data_dict, seq_len=60):
    X, y = [], []
    for t in data_dict.keys():
        data = scaled_data[t]
        for i in range(seq_len, len(data)):
            X.append(data[i - seq_len:i])
            y.append(data[i, 0])
    return np.array(X), np.array(y)

X, y = create_dataset(scaled_data, seq_len=seq_len)
split1, split2 = int(0.7 * len(X)), int(0.85 * len(X))

X_train, y_train = X[:split1], y[:split1]
X_val, y_val = X[split1:split2], y[split1:split2]
X_test, y_test = X[split2:], y[split2:]

print(f"📊 Dataset created:")
print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# -----------------------------------------------------------
# 4️⃣ Build the Updated LSTM Model
# -----------------------------------------------------------
from tensorflow.keras.optimizers import Adam

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(seq_len, X.shape[2])),
    BatchNormalization(),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='mean_squared_error')
model.summary()

# -----------------------------------------------------------
# 5️⃣ Callbacks for Stable Training
# -----------------------------------------------------------
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5),
    ModelCheckpoint(
        filepath="alphawave_vX4_checkpoints/best_model.keras",
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
]

# -----------------------------------------------------------
# 6️⃣ Train the Model
# -----------------------------------------------------------
print("\n🚀 Training AlphaWave vX.4 with updated data...\n")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

# -----------------------------------------------------------
# 7️⃣ Evaluate and Compare
# -----------------------------------------------------------
train_pred = model.predict(X_train)
val_pred = model.predict(X_val)
test_pred = model.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))

print("\n========== vX.4 RMSE Summary ==========")
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Validation RMSE: {val_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

# -----------------------------------------------------------
# 8️⃣ Save Final Model and Metrics
# -----------------------------------------------------------
model.save("alphawave_vX4_checkpoints/AlphaWave_vX4.keras")

metrics = pd.DataFrame({
    'Metric': ['Train_RMSE', 'Val_RMSE', 'Test_RMSE'],
    'Value': [train_rmse, val_rmse, test_rmse]
})
metrics.to_csv("alphawave_vX4_checkpoints/metrics_vX4.csv", index=False)

print("\n💾 Model and metrics saved to 'alphawave_vX4_checkpoints/'")
print("🎯 AlphaWave vX.4 retraining completed successfully.")


In [ ]:
# Example: Predict and plot for AAPL
t = "AAPL"
df = data_dict[t]
scaled = scalers[t].transform(df[['Close', 'Volume']])
X_stock = np.array([scaled[i - seq_len:i] for i in range(seq_len, len(scaled))])
preds = model.predict(X_stock, verbose=0)

plt.figure(figsize=(10,5))
plt.plot(df.index[seq_len:], scaled[seq_len:, 0], label='Actual (Scaled)', color='blue')
plt.plot(df.index[seq_len:], preds, label='Predicted', color='red')
plt.title(f"{t} — AlphaWave vX.4 Stock Prediction")
plt.xlabel('Date')
plt.ylabel('Standardized Close Price')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Normalize index lengths for plotting
train_range = range(len(train_pred))
val_range = range(len(train_pred), len(train_pred) + len(val_pred))
test_range = range(len(train_pred) + len(val_pred),
                    len(train_pred) + len(val_pred) + len(test_pred))

plt.figure(figsize=(12,6))
plt.plot(train_range, y_train, label='Train Actual', color='blue', alpha=0.4)
plt.plot(train_range, train_pred, label='Train Predicted', color='cyan', alpha=0.6)
plt.plot(val_range, y_val, label='Val Actual', color='green', alpha=0.4)
plt.plot(val_range, val_pred, label='Val Predicted', color='lime', alpha=0.6)
plt.plot(test_range, y_test, label='Test Actual', color='red', alpha=0.4)
plt.plot(test_range, test_pred, label='Test Predicted', color='orange', alpha=0.6)

plt.title("AlphaWave vX.4 — Training vs Validation vs Test Prediction")
plt.xlabel("Sequence Index")
plt.ylabel("Standardized Price")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Validation RMSE: {val_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")


In [ ]:
# ===========================================================
# AlphaWave vX.3 Generalization & Future Data Evaluation
# ===========================================================

from datetime import datetime
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import yfinance as yf

# -----------------------------------------------------------
# 1️⃣ Train vs Validation RMSE Ratio (Overfitting Metric)
# -----------------------------------------------------------

print("========== Train vs Validation RMSE Ratio ==========")

try:
    train_rmse_ratio = train_rmse / val_rmse
    print(f"Train RMSE:       {train_rmse:.4f}")
    print(f"Validation RMSE:  {val_rmse:.4f}")
    print(f"RMSE Ratio:       {train_rmse_ratio:.3f}")

    if train_rmse_ratio < 1.1:
        print("✅ Excellent generalization — model not overfitting.")
    elif 1.1 <= train_rmse_ratio <= 1.3:
        print("⚖️  Mild overfitting, acceptable range.")
    else:
        print("⚠️  Possible overfitting — model fits training data too tightly.")
except NameError:
    print("⚠️ RMSE variables not found. Skipping this part.")
    train_rmse_ratio = None

# -----------------------------------------------------------
# 2️⃣ Evaluate on Unseen 2025 Data (Future Drift Test)
# -----------------------------------------------------------

print("\n========== Future Data Evaluation (2025 onward) ==========")

future_results = []

for t in tickers:
    print(f"\nEvaluating {t}...")
    try:
        # Download unseen data
        future_data = yf.download(t, start='2025-01-01', end=datetime.today().strftime('%Y-%m-%d'))
        if len(future_data) < 60:
            print(f"⚠️  Not enough 2025 data for {t}, skipping.")
            continue

        # Preprocess using saved scaler
        scaled_future = scalers[t].transform(future_data[['Close', 'Volume']])
        
        # Create sequences
        X_future, y_future = [], []
        for i in range(seq_len, len(scaled_future)):
            X_future.append(scaled_future[i - seq_len:i])
            y_future.append(scaled_future[i, 0])
        X_future, y_future = np.array(X_future), np.array(y_future)

        # Predict & evaluate
        future_pred = model.predict(X_future, verbose=0)
        future_rmse = np.sqrt(mean_squared_error(y_future, future_pred))
        future_results.append({'Stock': t, 'Future_RMSE': future_rmse})

    except Exception as e:
        print(f"❌ Error evaluating {t}: {e}")

# Create summary DataFrame
future_df = pd.DataFrame(future_results)
print("\n========== Future RMSE Summary ==========")
print(future_df)

# -----------------------------------------------------------
# 3️⃣ Compare test vs future RMSE (Model Drift)
# -----------------------------------------------------------

if not future_df.empty:
    try:
        avg_future_rmse = future_df['Future_RMSE'].mean()
        print("\n========== Model Drift Summary ==========")
        print(f"Average Test RMSE (from training):  {test_rmse:.4f}")
        print(f"Average Future RMSE (2025 unseen):  {avg_future_rmse:.4f}")
        drift_ratio = avg_future_rmse / test_rmse
        print(f"RMSE Drift Ratio: {drift_ratio:.3f}")

        if drift_ratio < 1.2:
            print("✅ Model generalizes stably across unseen 2025 data.")
        elif 1.2 <= drift_ratio <= 1.5:
            print("⚖️  Moderate drift — retraining in 6 months recommended.")
        else:
            print("⚠️  Significant drift — consider retraining with recent data.")
    except NameError:
        print("⚠️ Test RMSE not found. Only showing future RMSEs.")
else:
    print("⚠️ No valid results for future data evaluation.")


In [ ]:
# AlphaWave vX.5 – Optimized Universal LSTM (Overfitting Controlled)

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import math
from sklearn.metrics import mean_squared_error

# ===============================
# 1. Load Multiple Stock Data
# ===============================
tickers = [
    'AAPL', 'MSFT', 'AMZN', 'GOOG', 'NVDA', 'META', 'TSLA', 'AMD',
    'ADBE', 'NFLX', 'INTC', 'IBM', 'PYPL', 'ORCL', 'CSCO', 'CRM',
    'QCOM', 'AVGO', 'TXN', 'SHOP', 'UBER', 'SNOW', 'BABA', 'PLTR', 'COIN'
]

data = yf.download(tickers, start='2018-01-01', end='2025-01-01')['Close']
data = data.fillna(method='ffill').dropna()

# ===============================
# 2. Data Preprocessing
# ===============================
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

lookback = 60
X, y = [], []

for i in range(lookback, len(scaled_data)):
    X.append(scaled_data[i-lookback:i])
    y.append(scaled_data[i])

X, y = np.array(X), np.array(y)

# Split data (temporal)
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)

X_train, X_val, X_test = (
    X[:train_size],
    X[train_size:train_size + val_size],
    X[train_size + val_size:]
)
y_train, y_val, y_test = (
    y[:train_size],
    y[train_size:train_size + val_size],
    y[train_size + val_size:]
)

# ===============================
# 3. Build Optimized LSTM Model
# ===============================
model = Sequential([
    LSTM(48, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]),
         dropout=0.3, recurrent_dropout=0.2, kernel_regularizer=l2(0.001)),
    LSTM(32, return_sequences=False, dropout=0.3, recurrent_dropout=0.2,
         kernel_regularizer=l2(0.001)),
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(y_train.shape[1])  # Predict all stocks simultaneously
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# ===============================
# 4. Train with EarlyStopping
# ===============================
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)

# ===============================
# 5. Evaluate Performance
# ===============================
train_pred = model.predict(X_train)
val_pred = model.predict(X_val)
test_pred = model.predict(X_test)

train_rmse = math.sqrt(mean_squared_error(y_train, train_pred))
val_rmse = math.sqrt(mean_squared_error(y_val, val_pred))
test_rmse = math.sqrt(mean_squared_error(y_test, test_pred))

print(f"Train RMSE: {train_rmse:.4f}")
print(f"Validation RMSE: {val_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

# ===============================
# 6. Plot Train vs Validation vs Test Predictions
# ===============================
plt.figure(figsize=(12, 6))
plt.plot(y_train.flatten()[:1000], label='Train Actual', color='blue', alpha=0.4)
plt.plot(train_pred.flatten()[:1000], label='Train Predicted', color='cyan', alpha=0.4)
plt.plot(y_val.flatten()[:500], label='Val Actual', color='green', alpha=0.4)
plt.plot(val_pred.flatten()[:500], label='Val Predicted', color='lime', alpha=0.4)
plt.plot(y_test.flatten()[:500], label='Test Actual', color='orange', alpha=0.5)
plt.plot(test_pred.flatten()[:500], label='Test Predicted', color='red', alpha=0.5)
plt.title("AlphaWave vX.5 — Training vs Validation vs Test Prediction")
plt.xlabel("Sequence Index")
plt.ylabel("Standardized Price")
plt.legend()
plt.show()

# ===============================
# 7. Save Model and Scaler
# ===============================
model.save("alphawave_vX5_optimized.keras")
import pickle
with open("scaler_vX5.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("✅ Model and scaler saved as 'alphawave_vX5_optimized.keras' and 'scaler_vX5.pkl'.")


In [ ]:
# ===========================================
# AlphaWave vX.6 — Stock-Aware GRU Hybrid
# ===========================================
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, GRU, Dense, Dropout, Concatenate, Flatten
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import math
import pickle

# ===========================================
# 1. Load and Prepare Multi-Stock Data
# ===========================================
tickers = [
    'AAPL', 'MSFT', 'AMZN', 'GOOG', 'NVDA', 'META', 'TSLA', 'AMD',
    'ADBE', 'NFLX', 'INTC', 'IBM', 'PYPL', 'ORCL', 'CSCO', 'CRM',
    'QCOM', 'AVGO', 'TXN', 'SHOP', 'UBER', 'SNOW', 'BABA', 'PLTR', 'COIN'
]

data = yf.download(tickers, start='2018-01-01', end='2025-01-01')['Close']
data = data.fillna(method='ffill').dropna()

# ===========================================
# 2. Per-Stock Normalization (Independent Scaling)
# ===========================================
scalers = {}
scaled_data = pd.DataFrame(index=data.index)

for col in data.columns:
    scaler = StandardScaler()
    scaled_data[col] = scaler.fit_transform(data[[col]])
    scalers[col] = scaler

lookback = 40
X, y, stock_ids = [], [], []

# Convert to sequences
for stock_idx, col in enumerate(data.columns):
    stock_series = scaled_data[col].values
    for i in range(lookback, len(stock_series)):
        X.append(stock_series[i-lookback:i])
        y.append(stock_series[i])
        stock_ids.append(stock_idx)

X = np.array(X)
y = np.array(y)
stock_ids = np.array(stock_ids)

# Reshape for GRU: (samples, timesteps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Train/Val/Test split
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)

X_train, X_val, X_test = X[:train_size], X[train_size:train_size+val_size], X[train_size+val_size:]
y_train, y_val, y_test = y[:train_size], y[train_size:train_size+val_size], y[train_size+val_size:]
stock_train, stock_val, stock_test = stock_ids[:train_size], stock_ids[train_size:train_size+val_size], stock_ids[train_size+val_size:]

# ===========================================
# 3. Build Stock-Aware GRU Model
# ===========================================
num_stocks = len(tickers)
embedding_dim = 8  # Compact but powerful stock representation

# Inputs
price_input = Input(shape=(lookback, 1), name="price_sequence")
stock_input = Input(shape=(1,), name="stock_id")

# Embedding Layer
embedding = Embedding(input_dim=num_stocks, output_dim=embedding_dim, name="stock_embedding")(stock_input)
embedding_flat = Flatten()(embedding)

# GRU Stack

gru_out = GRU(48, return_sequences=True, dropout=0.2, kernel_regularizer=l2(0.001))(price_input)
gru_out = GRU(32, return_sequences=False, dropout=0.2, kernel_regularizer=l2(0.001))(gru_out)


# Concatenate stock embedding + GRU output
merged = Concatenate()([gru_out, embedding_flat])

# Dense Layers
dense_out = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(merged)
dense_out = Dropout(0.3)(dense_out)
output = Dense(1, activation='linear')(dense_out)

# Compile Model
model = Model(inputs=[price_input, stock_input], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

model.summary()

# ===========================================
# 4. Train with EarlyStopping + LR Scheduler
# ===========================================
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

history = model.fit(
    [X_train, stock_train], y_train,
    validation_data=([X_val, stock_val], y_val),
    epochs=50,
    batch_size=64,
    callbacks=[early_stop, lr_scheduler],
    verbose=1
)

# ===========================================
# 5. Evaluate and Plot
# ===========================================
train_pred = model.predict([X_train, stock_train])
val_pred = model.predict([X_val, stock_val])
test_pred = model.predict([X_test, stock_test])

train_rmse = math.sqrt(mean_squared_error(y_train, train_pred))
val_rmse = math.sqrt(mean_squared_error(y_val, val_pred))
test_rmse = math.sqrt(mean_squared_error(y_test, test_pred))

print(f"Train RMSE: {train_rmse:.4f}")
print(f"Validation RMSE: {val_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

# Plot results
plt.figure(figsize=(12,6))
plt.plot(y_test[:800], label='Actual', color='blue', alpha=0.6)
plt.plot(test_pred[:800], label='Predicted', color='red', alpha=0.6)
plt.title("AlphaWave vX.6 — Stock-Aware GRU Predictions")
plt.xlabel("Time Steps")
plt.ylabel("Standardized Price")
plt.legend()
plt.show()

# ===========================================
# 6. Save Model and Scalers
# ===========================================
model.save("alphawave_vX6_stockaware.keras")

with open("scalers_vX6.pkl", "wb") as f:
    pickle.dump(scalers, f)

print("✅ Model and scalers saved successfully as 'alphawave_vX6_stockaware.keras' and 'scalers_vX6.pkl'.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_stock_predictions_and_forecast(stock_symbol, model, data, scalers, lookback=40, forecast_horizon=30):
    """
    Visualizes past predictions vs actual data and 30-day forecast for an existing trained multi-input GRU model.
    Compatible with models taking [price_sequence, stock_id].
    """
    # --- Step 1: Prepare data for the selected stock ---
    prices = data[stock_symbol].values.reshape(-1, 1)
    scaler = scalers[stock_symbol]
    scaled_prices = scaler.transform(prices)

    # Create sequences (X_test, y_test)
    X_test, y_test = [], []
    for i in range(lookback, len(scaled_prices)):
        X_test.append(scaled_prices[i - lookback:i])
        y_test.append(scaled_prices[i])
    X_test, y_test = np.array(X_test), np.array(y_test)

    # Encode stock ID properly
    stock_index = list(data.columns).index(stock_symbol)
    stock_input = np.full((len(X_test), 1), stock_index, dtype=np.int32)

    # --- Step 2: Predict full historical sequence ---
    # Ensure both inputs are proper numpy arrays with same batch dimension
    predicted_scaled = model.predict([X_test, stock_input], verbose=0)

    # --- Step 3: Inverse transform for readability ---
    actual_prices = scaler.inverse_transform(y_test)
    predicted_prices = scaler.inverse_transform(predicted_scaled)

    # --- Step 4: Plot Actual vs Predicted (Historical) ---
    plt.figure(figsize=(10, 5))
    plt.plot(actual_prices, color='blue', label='Actual')
    plt.plot(predicted_prices, color='red', label='Predicted')
    plt.title(f"{stock_symbol} Stock Prediction (AlphaWave vX.6)")
    plt.xlabel("Days")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.grid(True)
    plt.show()

    # --- Step 5: Forecast Next 30 Days ---
    last_seq = scaled_prices[-lookback:].reshape(1, lookback, 1)
    stock_input_future = np.array([[stock_index]], dtype=np.int32)
    future_preds_scaled = []

    for _ in range(forecast_horizon):
        next_pred_scaled = model.predict([last_seq, stock_input_future], verbose=0)[0, 0]
        future_preds_scaled.append(next_pred_scaled)
        # Update rolling window
        last_seq = np.append(last_seq[:, 1:, :], [[[next_pred_scaled]]], axis=1)

    future_preds_scaled = np.array(future_preds_scaled).reshape(-1, 1)
    future_prices = scaler.inverse_transform(future_preds_scaled)

    # --- Step 6: Plot Forecast ---
    plt.figure(figsize=(8, 4))
    plt.plot(range(1, forecast_horizon + 1), future_prices, marker='o', color='green')
    plt.title(f"{stock_symbol} - 30-Day Forecast (AlphaWave vX.6)")
    plt.xlabel("Days Ahead")
    plt.ylabel("Predicted Price (USD)")
    plt.grid(True)
    plt.show()

    print(f"Final predicted price after {forecast_horizon} days: ${future_prices[-1][0]:.2f}")

# ✅ Example usage
for ticker in ["AAPL", "TSLA", "GOOG", "AMZN"]:
    plot_stock_predictions_and_forecast(ticker, model, data, scalers)

